In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

C:\Users\haho6\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/",one_hot=True)

Instructions for updating:
Use the retry module or similar alternatives.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
total_epoch = 200
batch_size = 100
learning_rate = 0.01
n_hidden = 256
n_input = 28*28
n_noise = 128
n_class = 10

In [4]:
#Real data
X = tf.placeholder(tf.float32,[None,n_input])
#Noise data
Z = tf.placeholder(tf.float32,[None,n_noise])
#Y Hint Data
Y = tf.placeholder(tf.float32,[None,n_class])

In [5]:
def generater(noise, labels):
    with tf.variable_scope('Generater'):
        inputs = tf.concat([noise, labels],1)
        hidden = tf.layers.dense(inputs,n_hidden,tf.nn.relu)
        output = tf.layers.dense(hidden,n_input,activation=tf.nn.sigmoid)
        
        return output

In [6]:
def discriminaster(inputs, labels, reuse=None):
    with tf.variable_scope('Discriminaster') as scope:
        if(reuse):
            scope.reuse_variables()
            
        inputs = tf.concat([inputs, labels], 1)
        hidden = tf.layers.dense(inputs, n_hidden, activation=tf.nn.relu)
        output = tf.layers.dense(hidden , 1, activation=None)
            
        return output

In [7]:
def get_noise(batch_size, n_noise):
    return np.random.uniform(-1., 1., size = [batch_size, n_noise])

In [8]:
G = generater(Z, Y)

In [9]:
D_real = discriminaster(X, Y)
D_gene = discriminaster(G,Y,True)

In [10]:
loss_D_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_real,labels=tf.ones_like(D_real)))

loss_D_gene = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_gene,labels=tf.zeros_like(D_gene)))

In [11]:
loss_D = loss_D_gene + loss_D_real
loss_G = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_gene, labels=tf.ones_like(D_gene)))

In [12]:
vars_D = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='Discriminaster')
vars_G = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='Generater')

In [13]:
train_D = tf.train.AdamOptimizer().minimize(loss_D, var_list=vars_D)
train_G = tf.train.AdamOptimizer().minimize(loss_G, var_list=vars_G)

In [14]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [15]:
total_batch=int(mnist.train.num_examples / batch_size)
vars_D = 0
vars_G = 0

In [17]:
for epoch in range(total_epoch):
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        noise = get_noise(batch_size,n_noise)
        _, varsD=sess.run([train_D, loss_D], feed_dict={X:batch_xs, Y: batch_ys, Z: noise})
        _, varsG=sess.run([train_G, loss_G], feed_dict={X:batch_xs, Y: batch_ys, Z: noise})
        
        print('ep : ', epoch, 'D loss : ',varsD,'G loss',varsG)

ep :  0 D loss :  0.84650373 G loss 4.7129827
ep :  0 D loss :  0.68277776 G loss 5.116146
ep :  0 D loss :  0.4616941 G loss 5.021591
ep :  0 D loss :  0.2856014 G loss 4.740929
ep :  0 D loss :  0.175989 G loss 4.39612
ep :  0 D loss :  0.121071965 G loss 4.0725183
ep :  0 D loss :  0.08301909 G loss 3.9869843
ep :  0 D loss :  0.06481862 G loss 4.071976
ep :  0 D loss :  0.058105998 G loss 4.1782427
ep :  0 D loss :  0.043250777 G loss 4.460563
ep :  0 D loss :  0.04248993 G loss 4.604667
ep :  0 D loss :  0.030099884 G loss 4.7838893
ep :  0 D loss :  0.025355853 G loss 4.967584
ep :  0 D loss :  0.02384969 G loss 4.928079
ep :  0 D loss :  0.02301959 G loss 5.1063046
ep :  0 D loss :  0.025233284 G loss 4.8695283
ep :  0 D loss :  0.020525087 G loss 5.0435605
ep :  0 D loss :  0.029012378 G loss 4.784722
ep :  0 D loss :  0.03490248 G loss 4.776954
ep :  0 D loss :  0.027031036 G loss 4.903343
ep :  0 D loss :  0.035451356 G loss 4.6663103
ep :  0 D loss :  0.031069517 G loss 4.81

ep :  0 D loss :  0.035533555 G loss 5.691936
ep :  0 D loss :  0.029126484 G loss 5.6566443
ep :  0 D loss :  0.044951804 G loss 5.5589967
ep :  0 D loss :  0.033161443 G loss 5.5853667
ep :  0 D loss :  0.037005585 G loss 5.6565037
ep :  0 D loss :  0.047195587 G loss 5.62846
ep :  0 D loss :  0.018209465 G loss 5.7150764
ep :  0 D loss :  0.017616985 G loss 5.779803
ep :  0 D loss :  0.024334397 G loss 5.8547087
ep :  0 D loss :  0.019103434 G loss 5.9086213
ep :  0 D loss :  0.019468784 G loss 5.936855
ep :  0 D loss :  0.021655057 G loss 5.864622
ep :  0 D loss :  0.016451083 G loss 5.9237237
ep :  0 D loss :  0.0267561 G loss 5.8832026
ep :  0 D loss :  0.05363362 G loss 5.7676044
ep :  0 D loss :  0.031095617 G loss 5.621743
ep :  0 D loss :  0.023331255 G loss 5.6885486
ep :  0 D loss :  0.015926925 G loss 5.783502
ep :  0 D loss :  0.031520985 G loss 5.816633
ep :  0 D loss :  0.020428438 G loss 5.9185696
ep :  0 D loss :  0.028278219 G loss 5.9044757
ep :  0 D loss :  0.02060

ep :  0 D loss :  0.031955007 G loss 6.359255
ep :  0 D loss :  0.020647615 G loss 6.3976717
ep :  0 D loss :  0.026173668 G loss 6.439056
ep :  0 D loss :  0.020903293 G loss 6.5871096
ep :  0 D loss :  0.013470011 G loss 6.6380672
ep :  0 D loss :  0.019779257 G loss 6.5784807
ep :  0 D loss :  0.021109242 G loss 6.7017827
ep :  0 D loss :  0.015847322 G loss 6.912129
ep :  0 D loss :  0.017095001 G loss 6.8452997
ep :  0 D loss :  0.01381048 G loss 6.918032
ep :  0 D loss :  0.018159397 G loss 6.7683563
ep :  0 D loss :  0.01868397 G loss 6.6536474
ep :  0 D loss :  0.016275618 G loss 6.495882
ep :  0 D loss :  0.01701887 G loss 6.570372
ep :  0 D loss :  0.011134768 G loss 6.9497375
ep :  0 D loss :  0.02757306 G loss 6.6210036
ep :  0 D loss :  0.014877524 G loss 6.854481
ep :  0 D loss :  0.018597648 G loss 6.845
ep :  0 D loss :  0.019261695 G loss 6.9705744
ep :  0 D loss :  0.010290319 G loss 7.2473497
ep :  0 D loss :  0.01160183 G loss 7.4293547
ep :  0 D loss :  0.017181423

ep :  1 D loss :  0.010741359 G loss 7.9542456
ep :  1 D loss :  0.0037502502 G loss 7.726353
ep :  1 D loss :  0.007809846 G loss 7.5589647
ep :  1 D loss :  0.01006094 G loss 7.6715503
ep :  1 D loss :  0.0062201004 G loss 7.4926653
ep :  1 D loss :  0.008940617 G loss 7.625946
ep :  1 D loss :  0.005859228 G loss 7.9255395
ep :  1 D loss :  0.0037665458 G loss 7.9404783
ep :  1 D loss :  0.008771449 G loss 7.9592495
ep :  1 D loss :  0.00619254 G loss 7.948366
ep :  1 D loss :  0.012702163 G loss 7.9446135
ep :  1 D loss :  0.010937367 G loss 7.556634
ep :  1 D loss :  0.005768223 G loss 7.3579383
ep :  1 D loss :  0.0043699085 G loss 7.23172
ep :  1 D loss :  0.018634291 G loss 7.132349
ep :  1 D loss :  0.010556205 G loss 7.277962
ep :  1 D loss :  0.004081781 G loss 7.20549
ep :  1 D loss :  0.007763771 G loss 7.1632214
ep :  1 D loss :  0.00440289 G loss 7.5273805
ep :  1 D loss :  0.0065516895 G loss 7.724934
ep :  1 D loss :  0.0063351225 G loss 7.6272206
ep :  1 D loss :  0.0

ep :  1 D loss :  0.010804222 G loss 16.561232
ep :  1 D loss :  0.006365736 G loss 16.249622
ep :  1 D loss :  0.008843718 G loss 16.58443
ep :  1 D loss :  0.00867177 G loss 15.561366
ep :  1 D loss :  0.0069699395 G loss 14.425704
ep :  1 D loss :  0.009234425 G loss 15.293898
ep :  1 D loss :  0.00672312 G loss 13.846566
ep :  1 D loss :  0.0105028385 G loss 14.161278
ep :  1 D loss :  0.0061363587 G loss 13.419922
ep :  1 D loss :  0.005997535 G loss 14.02432
ep :  1 D loss :  0.004856351 G loss 12.709804
ep :  1 D loss :  0.006605791 G loss 12.930681
ep :  1 D loss :  0.0063985605 G loss 12.807313
ep :  1 D loss :  0.0051512704 G loss 12.548938
ep :  1 D loss :  0.006592094 G loss 12.289541
ep :  1 D loss :  0.0036616228 G loss 12.024128
ep :  1 D loss :  0.0028542746 G loss 12.986815
ep :  1 D loss :  0.0057421485 G loss 12.329022
ep :  1 D loss :  0.004761523 G loss 11.371471
ep :  1 D loss :  0.0034397908 G loss 12.126737
ep :  1 D loss :  0.0030184714 G loss 11.66557
ep :  1 

...

ep :  63 D loss :  0.6064684 G loss 2.1861951
ep :  63 D loss :  0.673317 G loss 2.1697488
ep :  63 D loss :  0.64791226 G loss 2.138404
ep :  63 D loss :  0.79069364 G loss 2.12006
ep :  63 D loss :  0.66707873 G loss 2.2091148
ep :  63 D loss :  0.7204427 G loss 2.2849233
ep :  63 D loss :  0.6992004 G loss 2.3108757
ep :  63 D loss :  0.702456 G loss 2.0443177
ep :  63 D loss :  0.69505715 G loss 1.9809647
ep :  63 D loss :  0.76373684 G loss 2.0772102
ep :  63 D loss :  0.68223846 G loss 2.2546425
ep :  63 D loss :  0.73886573 G loss 2.2059433
ep :  63 D loss :  0.7072283 G loss 2.0491025
ep :  63 D loss :  0.600584 G loss 2.1916227
ep :  63 D loss :  0.5995274 G loss 2.344498
ep :  63 D loss :  0.64313877 G loss 2.343473
ep :  63 D loss :  0.76238835 G loss 2.2644758
ep :  63 D loss :  0.75247025 G loss 2.1139207
ep :  63 D loss :  0.7661575 G loss 1.7963122
ep :  63 D loss :  0.69887686 G loss 2.0659556
ep :  63 D loss :  0.7721883 G loss 1.9445262
ep :  63 D loss :  0.6535454 G 

ep :  63 D loss :  0.8520643 G loss 1.7956673
ep :  63 D loss :  0.7866021 G loss 1.7310436
ep :  63 D loss :  0.6868663 G loss 2.054699
ep :  63 D loss :  0.7024121 G loss 2.3515224
ep :  63 D loss :  0.86208594 G loss 2.0481946
ep :  63 D loss :  0.82771075 G loss 1.947241
ep :  63 D loss :  0.7617066 G loss 2.281261
ep :  63 D loss :  0.76019454 G loss 2.5491173
ep :  63 D loss :  0.5588918 G loss 2.485858
ep :  63 D loss :  0.680131 G loss 2.2626102
ep :  63 D loss :  0.7011975 G loss 2.2284396
ep :  63 D loss :  0.8386175 G loss 1.9964656
ep :  63 D loss :  0.621323 G loss 2.1022863
ep :  63 D loss :  0.6846175 G loss 1.9846381
ep :  63 D loss :  0.9324419 G loss 1.8329288
ep :  63 D loss :  0.823608 G loss 2.1164715
ep :  63 D loss :  0.63083506 G loss 2.3856678
ep :  63 D loss :  0.8401793 G loss 2.094146
ep :  63 D loss :  0.6974766 G loss 2.212863
ep :  63 D loss :  0.67690206 G loss 2.29921
ep :  63 D loss :  0.6533094 G loss 2.30821
ep :  63 D loss :  0.5399987 G loss 2.4081

ep :  64 D loss :  0.7939854 G loss 2.0595725
ep :  64 D loss :  0.7697172 G loss 2.2844698
ep :  64 D loss :  0.6829697 G loss 2.486055
ep :  64 D loss :  0.63386416 G loss 2.626905
ep :  64 D loss :  0.7031357 G loss 2.250244
ep :  64 D loss :  0.9127837 G loss 2.597159
ep :  64 D loss :  0.8585049 G loss 2.3834214
ep :  64 D loss :  0.67390704 G loss 2.0309708
ep :  64 D loss :  0.5365492 G loss 2.102355
ep :  64 D loss :  0.8827741 G loss 1.8794113
ep :  64 D loss :  0.69121563 G loss 1.8978837
ep :  64 D loss :  0.7046298 G loss 2.237653
ep :  64 D loss :  0.7186276 G loss 2.2384343
ep :  64 D loss :  0.6214986 G loss 2.500322
ep :  64 D loss :  0.78004277 G loss 2.5341353
ep :  64 D loss :  0.58940005 G loss 2.6407385
ep :  64 D loss :  0.7363196 G loss 2.4051445
ep :  64 D loss :  0.698469 G loss 2.278786
ep :  64 D loss :  0.527778 G loss 2.3581638
ep :  64 D loss :  0.6970118 G loss 2.2330139
ep :  64 D loss :  0.6874878 G loss 2.015716
ep :  64 D loss :  0.71016294 G loss 2.1

ep :  64 D loss :  0.5909793 G loss 2.1916773
ep :  64 D loss :  0.53805304 G loss 2.5104585
ep :  64 D loss :  0.57715726 G loss 2.521496
ep :  64 D loss :  0.7784318 G loss 2.0521863
ep :  64 D loss :  0.8685975 G loss 2.128665
ep :  64 D loss :  0.8650895 G loss 1.8996263
ep :  64 D loss :  0.7449186 G loss 2.1634674
ep :  64 D loss :  0.7930618 G loss 2.1931074
ep :  64 D loss :  0.76120114 G loss 2.0948446
ep :  64 D loss :  0.6922132 G loss 2.2334235
ep :  64 D loss :  0.7488657 G loss 2.0957203
ep :  64 D loss :  0.77073526 G loss 2.0314894
ep :  64 D loss :  0.8967217 G loss 1.9619362
ep :  64 D loss :  0.83500385 G loss 1.8265797
ep :  64 D loss :  0.90263987 G loss 1.9720693
ep :  64 D loss :  0.79334897 G loss 1.8722881
ep :  64 D loss :  0.7889322 G loss 2.0628953
ep :  64 D loss :  0.7611065 G loss 2.153951
ep :  64 D loss :  0.759701 G loss 1.8889583
ep :  64 D loss :  0.69525576 G loss 2.054991
ep :  64 D loss :  0.7900263 G loss 2.2427692
ep :  64 D loss :  0.7247851 G 

KeyboardInterrupt: 

In [27]:
noise = get_noise(10,n_noise)
sess.run(G, feed_dict={Y:np.arange(0,10,1).reshape(10,1),  Z: noise})

ValueError: Cannot feed value of shape (10, 1) for Tensor 'Placeholder_2:0', which has shape '(?, 10)'